In [5]:
import pandas as pd
import numpy as np
import os, requests
from bs4 import BeautifulSoup as bs

Importing the necessary libraries

In [6]:
# The code was removed by Watson Studio for sharing.

URL from wikipedia

In [7]:
version = '20181002'
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Parsing data from the URL

In [8]:
wiki_data = requests.get(wiki_url)
wiki = bs(wiki_data.content, 'lxml')
#print(wiki)

Converting the HTML code into Dataframe

In [9]:
table_body = wiki.find('tbody')
table_rows = table_body.find_all('tr')
results = [[cell.text.strip() for cell in row.find_all('td')] for row in table_rows[1:] ]
df = pd.DataFrame(results,columns=["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Cleaning the dataframe

In [10]:
df = df[df.Borough != 'Not assigned']
mask = df.Neighborhood == 'Not assigned'
df.loc[mask,'Neighborhood'] = df[mask]['Borough']
df = df.groupby(['PostalCode','Borough']).agg({'Neighborhood':lambda x: ', '.join(x)}).reset_index()
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Sorting the dataframe

In [11]:
df.sort_values(by='Borough', inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()



,PostalCode,Borough,Neighborhood
0,M4S,Central Toronto,Davisville
1,M5N,Central Toronto,Roselawn
2,M4R,Central Toronto,North Toronto West
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"


Size of the data frame (number of rows, number of columns)

In [12]:
df.shape

(103, 3)

Tried to make geocoder to work, but it was just too slown and unreliable so I'm importing the CSV file

In [13]:
#pip install geocoder
#import geocoder # import geocoder
# initialize your variable to None
#lat_lng_coords = None
#for index, postalcode in df.iterrows():
#    lat_lng_coords=None
#    print(postalcode['PostalCode'])
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postalcode['PostalCode']))
#      lat_lng_coords = g.latlng
#    postalcode['Latitude']=(lat_lng_coords[0])
#    postalcode['Longitude']=(lat_lng_coords[1])

csv_file = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv' 
data = pd.read_csv(csv_file) 

data.head()
 

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Matching the Postal Codes from the file with ones on the Data Frame

In [14]:

for i, row in data.iterrows():
    for index, n in df.iterrows():
        if row['Postal Code'] == n['PostalCode']:
            df.at[index,'Latitude'] = row['Latitude'] 
            df.at[index,'Longitude']=row['Longitude']
            
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


Creating the clusters

In [21]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install geocoder
import geocoder # import geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done

# All requested packages already installed.

  Using cached https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/bb/a24838832ba35baf52f32ab1a49b906b5f82fb7c76b2f6a7e35e140bac30/decorator-4.3.0-py2.py3-

In [23]:

address = 'Toronto, ON'
geolocator = Nominatim()
location = geolocator.geocode(address)
olatitude = location.latitude
olongitude = location.longitude
print('The geograpical coordinate of Toronto, On are {}, {}.'.format(latitude, longitude))

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, On are 43.653963, -79.387207.


Generating the map of Ontario in Canada

In [25]:
# create map of Manhattan using latitude and longitude values
map_ontario = folium.Map(location=[olatitude, olongitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ontario)  
    
map_ontario

Getting the mean coordinates between the boroughs to center the map around them!

In [43]:
mean_lat = df['Latitude'].mean()
mean_lon = df['Longitude'].mean()
print(mean_lat)
print(mean_lon)

43.70460773398059
-79.39715291165048


In [44]:

from sklearn.cluster import KMeans


kclusters = 5
toronto_grouped_clustering = df.drop(['Neighborhood','Borough','PostalCode'], 1)  #Only the Lat and long
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

Now Lets print the 5 clusters!

In [56]:
df['Cluster Labels'] = kmeans.labels_
map_clusters = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster: ' + str(cluster) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

Note that on Github the maps won't be displayed always! But feel free to run on watson studio on the link bellow!

https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/d5cbee20-754e-406f-91ad-a2a5c4c8fb39/view?access_token=45caa2a39304236d81df18764dc8014e855b3a6114412ae9c5986078793333e1